In [280]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

np.random.seed(7)

In [281]:

train = pd.read_csv('/Users/ahmed/Downloads/computerscience/Kaggle/geoai-ground-level-no2-estimation/geoai-ground/data/Train.csv') 
test = pd.read_csv('/Users/ahmed/Downloads/computerscience/Kaggle/geoai-ground-level-no2-estimation/geoai-ground/data/Test.csv')
test_id = test['ID_Zindi']
groups = train['ID']

# EDA

In [282]:
num_feats = train.select_dtypes(include=['float'])
cat_feats = train.select_dtypes(include=['object'])

for col in num_feats: 
    print(train[col].value_counts().head(4))

LAT
45.601585    1096
45.371005    1096
45.045825    1096
45.104075    1096
Name: count, dtype: int64
LON
11.903551    1096
11.840830    1096
12.060869    1096
11.553241    1096
Name: count, dtype: int64
Precipitation
0.000000    68723
6.744884        8
5.474710        8
3.088108        8
Name: count, dtype: int64
LST
305.18    52
308.96    50
307.96    50
307.06    48
Name: count, dtype: int64
AAI
-0.578419    4
-0.691745    4
-0.214139    4
-0.695799    4
Name: count, dtype: int64
CloudFraction
0.000000    7756
1.000000     348
0.000163       5
0.798943       4
Name: count, dtype: int64
NO2_strat
0.000056    335
0.000057    308
0.000055    301
0.000057    300
Name: count, dtype: int64
NO2_total
0.000102    973
0.000103    968
0.000105    964
0.000106    964
Name: count, dtype: int64
NO2_trop
0.000102    212
0.000106    203
0.000109    191
0.000104    188
Name: count, dtype: int64
TropopausePressure
14446.02105    156
16747.88684    154
19356.06402     62
13066.85806     42
Name: coun

In [283]:
num_feats = train.columns 
num_feattest = list(num_feats)
if 'GT_NO2' in num_feattest:
    num_feattest.remove('GT_NO2')

test[num_feattest]

,ID_Zindi,Date,ID,LAT,LON,Precipitation,LST,AAI,CloudFraction,NO2_strat,NO2_total,NO2_trop,TropopausePressure
0,ID_2MYNQS,1/1/19,PD03,45.289376,11.642394,3.277529,NaN,-0.313361,0.771456,0.000024,0.000075,NaN,14440.02819
1,ID_P4U5WU,1/1/19,TV03,45.836941,12.510362,0.000000,NaN,-0.229512,0.398208,0.000023,0.000120,NaN,14434.04790
2,ID_U4KWPK,1/1/19,X5561,45.582894,8.842165,0.000000,282.98,-0.470822,0.153694,0.000023,0.000171,0.000148,14427.42478
3,ID_QGSNTZ,1/1/19,X5953,45.131947,10.015742,1.928031,NaN,0.132952,0.756917,0.000024,0.000266,NaN,14443.09006
4,ID_GHSZ6K,1/1/19,X6701,45.186329,9.146666,0.000000,NaN,-0.198272,0.678858,0.000023,0.000149,NaN,14440.85840
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6571,ID_GUSXU9,12/31/21,TV03,45.836941,12.510362,0.000000,282.58,-0.013364,0.000000,0.000032,0.000135,0.000103,13060.46860
6572,ID_GMVEG1,12/31/21,X5561,45.582894,8.842165,0.000000,285.12,-0.412887,0.002098,0.000031,0.000201,0.000171,13056.11764
6573,ID_GD6HNP,12/31/21,X5953,45.131947,10.015742,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6574,ID_J7YW1Y,12/31/21,X6701,45.186329,9.146666,0.000000,NaN,-1.025128,0.476947,0.000031,0.000751,NaN,13063.22260


In [284]:
num_feats = [ 'LAT', 'LON', 'Precipitation', 'LST', 'AAI',
       'CloudFraction', 'NO2_strat', 'NO2_total', 'NO2_trop',
       'TropopausePressure', 'GT_NO2']

num_feattest= [ 'LAT', 'LON', 'Precipitation', 'LST', 'AAI',
       'CloudFraction', 'NO2_strat', 'NO2_total', 'NO2_trop',
       'TropopausePressure']

In [288]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GroupKFold
from lightgbm import LGBMRegressor
from lightgbm import LGBMRegressor

train = train[num_feats]
test = test[num_feattest]
train = train.ffill()
y = train['GT_NO2']
x =  train.drop(['GT_NO2'],axis=1)

# Assuming train and test datasets are already defined
# groups should be defined, for example:
# groups = train['ID']

model = LGBMRegressor(random_state=7)
n_splits = 4
cv = GroupKFold(n_splits=n_splits)

def validate(trainset, testset, target_col):
    # Train the model
    model.fit(trainset.drop(columns=target_col), trainset[target_col])
    
    # Predict the test set
    pred = model.predict(testset.drop(columns=target_col))
    
    # Print standard deviation of the actual test values
    print('std:', testset[target_col].std())
    
    # Compute the RMSE
    score = mean_squared_error(testset[target_col], pred, squared=False)
    print('score:', score)
    
    return score

stds = []
rmse = []

# Iterate over cross-validation splits
for train_idx, test_idx in cv.split(train.drop(columns='GT_NO2'), train['GT_NO2'], groups=groups):
    train_v, test_v = train.iloc[train_idx], train.iloc[test_idx]
    stds.append(test_v['GT_NO2'].std())
    rmse.append(validate(train_v, test_v, 'GT_NO2'))

# Output RMSE and standard deviations
print('RMSE:', np.array(rmse).mean())
print('RMSE std:', np.array(rmse).std())
print('Standard Deviations:', stds)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000702 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2160
[LightGBM] [Info] Number of data points in the train set: 64664, number of used features: 10
[LightGBM] [Info] Start training from score 25.512023
std: 14.964925878790867
score: 10.267010798688371
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000540 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2160
[LightGBM] [Info] Number of data points in the train set: 64664, number of used features: 10
[LightGBM] [Info] Start training from score 23.809929


/Users/ahmed/Downloads/computerscience/Kaggle/geoai-ground-level-no2-estimation/geoai-ground/venv-geoai-ground-level-no2-estimation/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


std: 18.88422227925358
score: 11.443973997000827
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000547 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2160
[LightGBM] [Info] Number of data points in the train set: 64664, number of used features: 10
[LightGBM] [Info] Start training from score 23.974859


/Users/ahmed/Downloads/computerscience/Kaggle/geoai-ground-level-no2-estimation/geoai-ground/venv-geoai-ground-level-no2-estimation/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


std: 18.133787803033616
score: 11.523395048091976
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000559 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2162
[LightGBM] [Info] Number of data points in the train set: 65760, number of used features: 10
[LightGBM] [Info] Start training from score 24.595891


/Users/ahmed/Downloads/computerscience/Kaggle/geoai-ground-level-no2-estimation/geoai-ground/venv-geoai-ground-level-no2-estimation/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


std: 15.317816264923987
score: 10.168276228205059
RMSE: 10.850664017996559
RMSE std: 0.6346038027983728
Standard Deviations: [np.float64(14.964925878790867), np.float64(18.88422227925358), np.float64(18.133787803033616), np.float64(15.317816264923987)]


/Users/ahmed/Downloads/computerscience/Kaggle/geoai-ground-level-no2-estimation/geoai-ground/venv-geoai-ground-level-no2-estimation/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [289]:
np.array(rmse).mean()

np.float64(10.850664017996559)

In [287]:
j[pkopk]

NameError: name 'j' is not defined

In [ ]:
from lightgbm import LGBMRegressor
train = train[num_feats]
test = test[num_feattest]

y = train['GT_NO2']
x =  train.drop(['GT_NO2'],axis=1)

"""le = LabelEncoder()
for df in (test,train): 
    for col in df.columns: 
        df[col]"""

#x_train, y_train, x_test,y_test = train_test_split(x,y,test_size=)
rfr = LGBMRegressor()
rfr.fit(x,y)
y_pred = rfr.predict(test)
sub_df = pd.DataFrame({'id': test_id,'GT_NO2':y_pred})
sub_df.to_csv('submissions.csv', index=False)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000924 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2200
[LightGBM] [Info] Number of data points in the train set: 86584, number of used features: 10
[LightGBM] [Info] Start training from score 24.473693
